In [1]:
import os,sys
import numpy as np
import logging
import sys,time
#sys.path.append('../')
os.chdir('../')
%load_ext line_profiler
from src.costs.full_shape_gradient import Full_shape_gradient

In [2]:
full_grad=Full_shape_gradient(path_config_file='config_file/config_full.ini')
init_param=full_grad.init_param

In [3]:
from src.costs.EM_cost import EM_cost_dask

In [4]:
%timeit full_grad.cost(init_param)

1 loop, best of 3: 4.74 s per loop


In [5]:
%lprun -u 1 -f EM_cost_dask full_grad.cost(init_param)

Timer unit: 1 s

Total time: 3.96789 s
File: /home/rrobin/Documents/Stellacage_code/src/costs/EM_cost.py
Function: EM_cost_dask at line 84

Line #      Hits         Time  Per Hit   % Time  Line Contents
    84                                           def EM_cost_dask(config,S,Sp):
    85                                               #new version without Lagrange multipliers
    86         1          0.0      0.0      0.0      import dask.array as da
    87                                               #initilization of the parameters
    88         1          0.0      0.0      0.0      lamb = float(config['other']['lamb'])
    89         1          0.0      0.0      0.0      Np = int(config['geometry']['Np'])
    90         1          0.0      0.0      0.0      ntheta_plasma = int(config['geometry']['ntheta_plasma'])
    91         1          0.0      0.0      0.0      ntheta_coil   = int(config['geometry']['ntheta_coil'])
    92         1          0.0      0.0      0.0      nzeta_pla

In [9]:
config['other']['dask']='True'
%lprun -u 1 -f cost_surface cost_surface(config)

Timer unit: 1 s

Total time: 6.82874 s
File: main/cost_surface.py
Function: cost_surface at line 9

Line #      Hits         Time  Per Hit   % Time  Line Contents
     9                                           def cost_surface(config,S=None,Sp=None):
    10         1          0.0      0.0      0.0      if config['other']['dask']=='True':
    11         1          6.8      6.8    100.0          return cost_surface_dask(config,S,Sp)
    12                                               else:
    13                                                   return cost_surface_without_dask(config,S,Sp)

In [5]:
%timeit cost_surface(config)

1 loop, best of 3: 12.9 s per loop


In [4]:
result={}
#same as regcoil param exept S which is given directly
#initilization of the parameters
lamb = regcoil_param.lamb
Np = regcoil_param.Np
path_plasma = regcoil_param.path_plasma
path_bnorm = regcoil_param.path_bnorm
net_poloidal_current_Amperes = regcoil_param.net_poloidal_current_Amperes
net_toroidal_current_Amperes = regcoil_param.net_toroidal_current_Amperes
ntheta_plasma = regcoil_param.ntheta_plasma
nzeta_plasma = regcoil_param.nzeta_plasma
phisize = regcoil_param.phisize
curpol= regcoil_param.curpol

#initialization of the plasma surface
Sp_parametrization=Toroidal_surface.load_file(path_plasma)
Sp=Toroidal_surface(Sp_parametrization,(ntheta_plasma,nzeta_plasma),Np)
#standard regcoil computations
rot_tensor=vector_field.get_rot_tensor(Np)
T=vector_field.get_tensor_distance(S,Sp,rot_tensor)
matrixd_phi=vector_field.get_matrix_dPhi(phisize,S.grids)
Qj=vector_field.compute_Qj(matrixd_phi,S.jac_xyz,S.ds)
array_bnorm=curpol*bnorm.get_bnorm(path_bnorm,Sp)
theta,dtildetheta,dtheta,dSdtheta=S.get_theta_pertubation()
dpsi=S.jac_xyz
normalp=Sp.n

In [5]:

_,lu1,lv1,_=matrixd_phi.shape
D=1/(np.linalg.norm(T,axis=-1)**3)
DD=1/(np.linalg.norm(T,axis=-1)**5)
#for cross product
eijk = np.zeros((3, 3, 3))
eijk[0, 1, 2] = eijk[1, 2, 0] = eijk[2, 0, 1] = 1
eijk[0, 2, 1] = eijk[2, 1, 0] = eijk[1, 0, 2] = -1
#the terrible formula...
#first dT/dtheta
#dLS=-contract('inb,ajkb,ojkw,ipz,wzjk,qnp,ijklm,qlm->aolm',rot_tensor,theta,matrixd_phi,rot_tensor,dpsi,eijk,D,normalp,optimize=True)/(lu1*lv1)
#then the -3(x-y,h)(x-y)

#dLS+=3*contract('ibc,ajkc,ijklmb,ijklmn,ojkw,ipz,wzjk,qnp,ijklm,qlm->aolm',rot_tensor,theta,T,T,matrixd_phi,rot_tensor,dpsi,eijk,D,normalp,optimize=True)/(lu1*lv1)
#finally the dpsi/dtheta part
#dLS=np.einsum('inb,ajkb,ojkw,ipz,wzjk,qnp,ijklm,qlm->aolm',rot_tensor,theta,matrixd_phi,rot_tensor,dpsi,eijk,D,normalp,optimize=True)

#dLS+=contract('ijklmn,ojkw,ipz,ajkbz,wbjk,qnp,ijklm,qlm->aolm',T,matrixd_phi,rot_tensor,dtildetheta,dpsi,eijk,D,normalp,optimize=True)/(lu1*lv1)

In [6]:
#%timeit contract('inb,ajkb,ojkw,ipz,wzjk,qnp,ijklm,qlm->aolm',rot_tensor,theta,matrixd_phi,rot_tensor,dpsi,eijk,D,normalp,optimize=True)/(lu1*lv1)

In [ ]:
t0=time.time()
dLS=contract('inb,ajkb,ojkw,ipz,wzjk,qnp,ijklm,qlm->aolm',rot_tensor,theta,matrixd_phi,rot_tensor,dpsi,eijk,D,normalp,optimize=True)/(lu1*lv1)
print(time.time()-t0)

In [ ]:
import cupy as cp
import time
import opt_einsum as oe

In [ ]:
mempool = cp.get_default_memory_pool()
pinned_mempool = cp.get_default_pinned_memory_pool()
mempool.free_all_blocks()
print('{:,}'.format(mempool.used_bytes()))
print('{:,}'.format(mempool.total_bytes()))
print('{:,}'.format(T.nbytes))
#print(dLS.shape,matrixd_phi.shape)

In [ ]:
chunk_lu=32
chunk_lv=32
chunk_lu_plasma=32
chunk_lv_plasma=32
chunk_theta=17
dask_rot_tensor = da.from_array(rot_tensor, asarray=False)
dask_theta=da.from_array(theta,chunks=(chunk_theta,chunk_lu,chunk_lv,3), asarray=False)
dask_matrixd_phi=da.from_array(matrixd_phi,chunks={1:chunk_lu,2:chunk_lv}, asarray=False)
dask_dpsi=da.from_array(dpsi,chunks=(2,3,chunk_lu,chunk_lv), asarray=False)
dask_eijk=da.from_array(eijk, asarray=False)
dask_normalp=da.from_array(normalp, asarray=False)
dask_D=da.from_array(D,chunks=(3,chunk_lu,chunk_lv,chunk_lu_plasma,chunk_lv_plasma), asarray=False)
dask_T=da.from_array(T,chunks=(3,chunk_lu,chunk_lv,chunk_lu_plasma,chunk_lv_plasma,3), asarray=False)
dask_dtildetheta=da.from_array(dtildetheta,chunks=(chunk_theta,chunk_lu,chunk_lv,3,3), asarray=False)


#theta_cupy=cp.asarray(theta)
#dpsi_cupy=cp.asarray(dpsi)
#eijk_cupy=cp.asarray(eijk)
#D_cupy=cp.asarray(D)
#normalp_cupy=cp.asarray(normalp)
#T_cupy=cp.asarray(T)

In [ ]:
#elts=[dask_rot_tensor,dask_theta,dask_matrixd_phi,dask_dpsi,dask_eijk,dask_D,dask_normalp,dask_T]
dtype=np.float32
dtype=np.float64

if False:
    dask_rot_tensor=dask_rot_tensor.map_blocks(cp.asarray,dtype=dtype)
    dask_theta=dask_theta.map_blocks(cp.asarray,dtype=dtype)
    dask_matrixd_phi=dask_matrixd_phi.map_blocks(cp.asarray,dtype=dtype)
    dask_dpsi=dask_dpsi.map_blocks(cp.asarray,dtype=dtype)
    dask_eijk=dask_eijk.map_blocks(cp.asarray,dtype=dtype)
    dask_D=dask_D.map_blocks(cp.asarray,dtype=dtype)
    dask_normalp=dask_normalp.map_blocks(cp.asarray,dtype=dtype)
    dask_T=dask_T.map_blocks(cp.asarray,dtype=dtype)
#for elt in elts:
#    elt = elt.map_blocks(cp.asarray)
#    print(elt)

In [ ]:
dask_theta

In [ ]:
first_expr = oe.contract_expression('inb,ajkb,ojkw,ipz,wzjk,qnp,ijklm,qlm->aolm',rot_tensor.shape,theta.shape,matrixd_phi.shape,rot_tensor.shape,dpsi.shape,eijk.shape,D.shape,normalp.shape)
#dask_exp1=first_expr(dask_rot_tensor,dask_theta,dask_matrixd_phi,dask_rot_tensor,dask_dpsi,dask_eijk,dask_D,dask_normalp)/(lu1*lv1)
dask_exp1=da.einsum('inb,ajkb,ojkw,ipz,wzjk,qnp,ijklm,qlm->aolm',dask_rot_tensor,dask_theta,dask_matrixd_phi,dask_rot_tensor,dask_dpsi,dask_eijk,dask_D,dask_normalp)/(lu1*lv1)
dask_exp2=first_expr(dask_rot_tensor,dask_theta,dask_matrixd_phi,dask_rot_tensor,dask_dpsi,dask_eijk,dask_D,dask_normalp)/(lu1*lv1)

In [ ]:
dLS_dask=-contract('inb,ajkb,ojkw,ipz,wzjk,qnp,ijklm,qlm->aolm',dask_rot_tensor,dask_theta,dask_matrixd_phi,dask_rot_tensor,dask_dpsi,dask_eijk,dask_D,dask_normalp,optimize=True)/(lu1*lv1)
#then the -3(x-y,h)(x-y)

dLS_dask+=3*contract('ibc,ajkc,ijklmb,ijklmn,ojkw,ipz,wzjk,qnp,ijklm,qlm->aolm',dask_rot_tensor,dask_theta,dask_T,dask_T,dask_matrixd_phi,dask_rot_tensor,dask_dpsi,dask_eijk,dask_D,dask_normalp,optimize=True)/(lu1*lv1)
#finally the dpsi/dtheta part
#dLS=np.einsum('inb,ajkb,ojkw,ipz,wzjk,qnp,ijklm,qlm->aolm',rot_tensor,theta,matrixd_phi,rot_tensor,dpsi,eijk,D,normalp,optimize=True)

dLS_dask+=contract('ijklmn,ojkw,ipz,ajkbz,wbjk,qnp,ijklm,qlm->aolm',dask_T,dask_matrixd_phi,dask_rot_tensor,dask_dtildetheta,dask_dpsi,dask_eijk,dask_D,dask_normalp,optimize=True)/(lu1*lv1)

In [ ]:
print('{:,}'.format(mempool.used_bytes()))
print('{:,}'.format(mempool.total_bytes()))
print('{:,}'.format(T.nbytes))
#dask_exp2.visualize(rankdir="LR")
print(first_expr)

In [ ]:
#dask_exp2.visualize(rankdir="LR")
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler

In [ ]:
t0=time.time()
with Profiler() as prof, ResourceProfiler(dt=0.25) as rprof, CacheProfiler() as cprof:
    dask_exp2.compute()
    #dask_exp1.compute()
    #dLS_dask.compute()
print(time.time()-t0)

In [18]:
from dask.diagnostics import visualize
visualize([prof, rprof, cprof])

Column(id='1252', ...)

In [19]:
number_chunk_theta=17
chunksize_theta=theta.shape[0]//number_chunk_theta # as to be an integer
theta_chunk=np.split(theta,number_chunk_theta)
#first dT/dtheta
first_expr = oe.contract_expression('inb,ajkb,ojkw,ipz,wzjk,qnp,ijklm,qlm->aolm',rot_tensor.shape,theta_chunk[0].shape,matrixd_phi.shape,rot_tensor.shape,dpsi.shape,eijk.shape,D.shape,normalp.shape)
#then the -3(x-y,h)(x-y)
second_expr= oe.contract_expression('ibc,ajkc,ijklmb,ijklmn,ojkw,ipz,wzjk,qnp,ijklm,qlm->aolm',rot_tensor.shape,theta_chunk[0].shape,T.shape,T.shape,matrixd_phi.shape,rot_tensor.shape,dpsi.shape,eijk.shape,D.shape,normalp.shape,)
print(second_expr)
t0=time.time()
dLS_cupy=cp.asarray(np.zeros((theta.shape[0],matrixd_phi.shape[0],D.shape[3],D.shape[4])))
for i in range(number_chunk_theta):
    #my_expr_cst(theta_chunk[i],backend='cupy')
    dLS_cupy[i*chunksize_theta:(i+1)*chunksize_theta]=first_expr(rot_tensor_cupy,cp.asarray(theta_chunk[i]),matrixd_phi_cupy,rot_tensor_cupy,dpsi_cupy,eijk_cupy,D_cupy,normalp_cupy,backend='cupy')/(lu1*lv1)
    #dLS_cupy[i*chunksize:(i+1)*chunksize]+=3*second_expr(rot_tensor_cupy,cp.asarray(theta_chunk[i]),T_cupy,T_cupy,matrixd_phi_cupy,rot_tensor_cupy,dpsi_cupy,eijk_cupy,D_cupy,normalp_cupy)/(lu1*lv1)
#dLS=contract('inb,ajkb,ojkw,ipz,wzjk,qnp,ijklm,qlm->aolm',rot_tensor_'optimal'cupy,theta_cupy,matrixd_phi_cupy,rot_tensor_cupy,dpsi_cupy,eijk_cupy,D,normalp_cupy,backend='cupy')/(lu1*lv1)
print(time.time()-t0)
print('{:,}'.format(mempool.used_bytes()))
print('{:,}'.format(mempool.total_bytes()))
print('{:,}'.format(T.nbytes))

<ContractExpression('ibc,ajkc,ijklmb,ijklmn,ojkw,ipz,wzjk,qnp,ijklm,qlm->aolm')>
  1.  'qnp,ipz->qniz' [TDOT]
  2.  'qniz,ibc->qnizbc'
  3.  'wzjk,ojkw->zjko'
  4.  'ijklm,ijklmb->ijklmb'
  5.  'qnizbc,qlm->nizbclm' [GEMM]
  6.  'zjko,ajkc->zjkoac'
  7.  'ijklmb,ijklmn->ijklmbn'
  8.  'ijklmbn,nizbclm->jklmzc'
  9.  'jklmzc,zjkoac->aolm' [TDOT]


NameError: name 'rot_tensor_cupy' is not defined

In [ ]:
print('{:,}'.format(mempool.used_bytes()))
print('{:,}'.format(mempool.total_bytes()))
print('{:,}'.format(T.nbytes))

In [ ]:
rot_tensor_cupy=cp.asarray(rot_tensor)
theta_cupy=cp.asarray(theta)
matrixd_phi_cupy=cp.asarray(matrixd_phi)
dpsi_cupy=cp.asarray(dpsi)
eijk_cupy=cp.asarray(eijk)
D_cupy=cp.asarray(D)
normalp_cupy=cp.asarray(normalp)
T_cupy=cp.asarray(T)